In [1]:
import os
nb_path = os.path.abspath("")
from sys import path
from os.path import dirname

path.insert(0,  dirname(nb_path))

nb_path

'c:\\Users\\ethan\\coding_projects\\trilogy-public-models\\local_examples'

In [2]:
from preql import Dialects
from trilogy_public_models import models
env = models["bigquery.thelook_ecommerce"]

executor = Dialects.BIGQUERY.default_executor(environment=env)

In [4]:
QA_3 = """

key cancelled_orders <- filter orders.id where orders.status = 'Cancelled';
auto orders.id.cancelled_count <- count(cancelled_orders);

SELECT
    orders.users.city,
    orders.id.cancelled_count / orders.id.count -> cancellation_rate,
    orders.id.cancelled_count,
    orders.id.count,
    orders.created_at.year,
WHERE
    (orders.created_at.year = 2020)
    and orders.id.count>10
ORDER BY
    cancellation_rate desc;
"""

results = executor.execute_text(QA_3)


In [5]:
QA_3 = """
SELECT 
    order_items.status,
    count(order_items.status)->count
;
"""

results = executor.execute_text(QA_3)
for row in results[0]:
    print(row)


('Cancelled', 27352)
('Complete', 45121)
('Shipped', 54127)
('Processing', 36160)
('Returned', 18056)


In [13]:
QA_3 = """
property order_items.id.revenue <- order_items.sale_price*order_items.orders.item_count;
# shipped, complete, processing
auto order_items.bookable_revenue <- filter order_items.revenue where order_items.status IN ( 'Shipped');
SELECT 
    order_items.orders.created_at.month,
    sum(order_items.bookable_revenue) -> revenue,
ORDER BY order_items.orders.created_at.month DESC;
"""

results = executor.execute_text(QA_3)


DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 200 Job execution was cancelled: User requested cancellation

Location: US
Job ID: 1e0f701c-00ac-4b7b-b703-93993f6020f4

[SQL: 
WITH 
civet as (
SELECT
    order_items_orders_orders.`created_at` as `order_items_orders_created_at`,
    order_items_orders_orders.`order_id` as `order_items_orders_id`,
    order_items_orders_orders.`num_of_item` as `order_items_orders_item_count`
FROM
    bigquery-public-data.thelook_ecommerce.orders as order_items_orders_orders
),
charming as (
SELECT
    EXTRACT(MONTH from civet.`order_items_orders_created_at`) as `order_items_orders_created_at_month`
FROM
    civet as civet
),
abundant as (
SELECT
    order_items_order_items.`sale_price` as `order_items_sale_price`,
    order_items_order_items.`order_id` as `order_items_orders_id`,
    order_items_order_items.`status` as `order_items_status`
FROM
    bigquery-public-data.thelook_ecommerce.order_items as order_items_order_items
),
exub

In [ ]:
for row in results[0]:
    print(row)

('Hamburg', 0.38461538461538464, 5, 13, 2020)
('Berlin', 0.3333333333333333, 10, 30, 2020)
('Zhongshan', 0.3333333333333333, 5, 15, 2020)
('Zhengzhou', 0.3269230769230769, 17, 52, 2020)
('Kunming', 0.3076923076923077, 4, 13, 2020)
('Yueyang', 0.3076923076923077, 4, 13, 2020)
('Rio de Janeiro', 0.30434782608695654, 7, 23, 2020)
('Bucheon City', 0.2727272727272727, 3, 11, 2020)
('Los Angeles', 0.2727272727272727, 9, 33, 2020)
('Jacksonville', 0.26666666666666666, 4, 15, 2020)
('San Antonio', 0.26666666666666666, 4, 15, 2020)
('Manaus', 0.2631578947368421, 5, 19, 2020)
('Guangzhou', 0.25, 13, 52, 2020)
('Dallas', 0.25, 3, 12, 2020)
('City of Yantai', 0.25, 3, 12, 2020)
("Lan'Zhou", 0.2413793103448276, 7, 29, 2020)
('Fort Worth', 0.23529411764705882, 4, 17, 2020)
('Paris', 0.23333333333333334, 7, 30, 2020)
('Frankfurt am Main', 0.23076923076923078, 3, 13, 2020)
('Fuzhou', 0.225, 9, 40, 2020)
('Urumqi', 0.21428571428571427, 6, 28, 2020)
('Brisbane', 0.21052631578947367, 4, 19, 2020)
('Shenz